In [28]:
%reset

In [30]:
import os ########################################
import numpy as np ###############################
import scipy.stats as stats ######################
import math ######################################
import matplotlib.pyplot as plt###################
from sklearn.metrics import mean_squared_error####

In [3]:
def create_Raw_hist(Raw_data, ensembleMenber, time):
    data = Raw_data
    min_of_data = data.min()
    max_of_data = data.max()
    bin_info = []
    temp_bin_info = []
    total_his_bins = 0
    if(len(data.shape) > 3):
        for timestep in range(time):
            timestep_bins = 0
            for x in range(data.shape[0]):
                for y in range(data.shape[1]):
                    for z in range(data.shape[2]):
                        rawdata = data[x, y, z, timestep*ensembleMenber:(timestep+1)*ensembleMenber]
                        hist = np.histogram(rawdata, bins=128, range=(min_of_data, max_of_data))
                        histprob = np.array(hist[0]/ensembleMenber)
                        histcount = np.array(hist[0])
                        total_his_bins += (len(histcount[histcount!=0]))
                        timestep_bins += (len(histcount[histcount!=0]))
                        grid_index = x*data.shape[1]*data.shape[2] + y*data.shape[2] + z
                        for bin_index in range(len(histprob)):
                            if(histprob[bin_index] == 0): continue
                            else:
                                binprob = histprob[bin_index]
                                bincount = float(histcount[bin_index])
                                temp_bin_info.append([timestep, grid_index, bin_index, bincount])
            # print('Bins of each time step: ', timestep_bins)
            # print('Num of bins: ', total_his_bins)
        bin_info = np.array(temp_bin_info)
                  
    else:
        for timestep in range(time):
            for x in range(data.shape[0]):
                for y in range(data.shape[1]):
                    rawdata = data[x, y, timestep*ensembleMenber:(timestep+1)*ensembleMenber]
                    hist = np.histogram(rawdata, bins=128, range=(min_of_data, max_of_data))
                    histprob = np.array(hist[0]/ensembleMenber)
                    histcount = np.array(hist[0])
                    grid_index = x*data.shape[1] + y
                    for bin_index in range(len(histprob)):
                        if(histprob[bin_index] == 0): continue 
                        else:
                            binprob = histprob[bin_index]
                            bincount = float(histcount[bin_index])
                            temp_bin_info.append([timestep, grid_index, bin_index, bincount])
        bin_info = np.array(temp_bin_info)
                
    return  bin_info

In [4]:
def reconstruct_raw_bin(bin_data, x, y, z, TotalTimesteps, NumOfBins, NumOfEnsemble):
    if(z != 0):
       Reconted_data = np.zeros((TotalTimesteps, x, y, z, NumOfBins))
       for bin in bin_data:
           timestep = int(bin[0])
           x_location = int(bin[1]//(y*z))
           y_location = int((bin[1]%(y*z))//z)
           z_location = int((bin[1]%(y*z))%z)
           bin_index = int(bin[2])
           bin_prob = bin[3]/NumOfEnsemble
           Reconted_data[timestep, x_location, y_location, z_location, bin_index] = bin_prob
    else:
        Reconted_data = np.zeros((int(bin_data[:,0].max()+1), x, y, NumOfBins))
        # Reconted_data = np.zeros((TotalTimesteps, x, y, NumOfBins))
        for bin in bin_data:
            timestep = int(bin[0])
            x_location = int(bin[1]//y)
            y_location = int(bin[1]%y)
            if(y_location>=128):
                bin_index = 127
            bin_index = int(bin[2])
            if(bin_index>=128):
                bin_index = 127
            bin_prob = bin[3]/NumOfEnsemble
            Reconted_data[timestep, x_location, y_location, bin_index] = bin_prob
    return Reconted_data

In [5]:
def compute_error(reconstruction, raw):
    data_size = reconstruction.shape
    if (len(reconstruction.shape) >3):
        data_x = data_size[0]
        data_y = data_size[1]
        data_z = data_size[2] 
        error = 0
        for x in range(data_x):
            for y in range(data_y):
                for z in range(data_z):
                    error += math.sqrt(mean_squared_error(reconstruction[x,y,z], raw[x,y,z]))
        rmse = error / (data_x*data_y*data_z) 
    else:
        data_x = data_size[0]
        data_y = data_size[1]
        error = 0
        for x in range(data_x):
            for y in range(data_y):
                error += math.sqrt(mean_squared_error(reconstruction[x,y], raw[x,y]))
        rmse = error / (data_x*data_y)
    return rmse     

In [ ]:
redsea_data = np.load('RedSea_25_100_100_ensemble50_timestep60.npy') # Re conduct
Redsea_raw_bin_data_count = create_Raw_hist(redsea_data, 50, 60)
Redsea_raw_bin_data_count.astype('float32').tofile('Redsea_raw_bin_data_count.bin')
Redsea_reconted_raw = reconstruct_raw_bin(Redsea_raw_bin_data_count, 25, 100, 100, 60, 128, 50)
Redsea_reconted_raw.astype('float32').tofile('RedSea_raw_reconted_60_25_100_100_128.bin')

In [18]:
redsea_bin_data = np.fromfile('Redsea_raw_bin_data_count.bin', dtype='float32').reshape((16507488,4))

In [20]:
Redsea_reconted_raw = reconstruct_raw_bin(redsea_bin_data, 25, 100, 100, 60, 128, 50)

In [25]:
Redsea_reconted_raw.astype('float32').tofile('redsea_raw_reconted_60_25_100_100_128.bin')

In [27]:
Redsea_reconted_raw.shape

(60, 25, 100, 100, 128)

In [6]:
nyx_data = np.load('nyx_density_32cube_ensemble64_timestep200.npy')
nyx_32cube_raw_bin_data = create_Raw_hist(nyx_data,64,200)
nyx_32cube_raw_bin_data.astype('float32').tofile('nyx_32cube_raw_bin_data_count.bin')
nyx_reconted_raw = reconstruct_raw_bin(nyx_32cube_raw_bin_data ,32 ,32, 32, 200, 128, 64)
nyx_reconted_raw.astype('float32').tofile('nyx_32cube_raw_reonted_200_32_32_32_128.bin')

In [162]:
sz_bindata = np.fromfile('air_raw_bin_data.bin.sz.out',dtype='float32').reshape((8010121,4))

In [163]:
sz_reconted = reconstruct_raw_bin(sz_bindata, 64, 128,0 ,150, 128)

In [164]:
sz_error = np.zeros(150)
for i in range(150):
    sz_error[i] = compute_error(sz_reconted[i], test_reconted_raw[i])

In [3]:
nyx_bin_data = np.fromfile('nyx_48cube_raw_bin_data_count.bin',dtype='float32')

In [6]:
nyx_bin_data = nyx_bin_data.reshape((22118400,4))

In [12]:
nyx_bin_data

array([[0.00000e+00, 0.00000e+00, 1.27000e+02, 0.00000e+00],
       [0.00000e+00, 1.00000e+00, 1.27000e+02, 0.00000e+00],
       [0.00000e+00, 2.00000e+00, 1.27000e+02, 0.00000e+00],
       ...,
       [1.99000e+02, 1.10589e+05, 1.27000e+02, 0.00000e+00],
       [1.99000e+02, 1.10590e+05, 1.27000e+02, 0.00000e+00],
       [1.99000e+02, 1.10591e+05, 1.27000e+02, 0.00000e+00]],
      dtype=float32)

In [31]:
sz_test = np.fromfile('air_raw_bin_data_count.bin.sz.out',dtype='float32')

In [33]:
sz = sz_test.reshape((11233538,4))